In [4]:
from deepeval.synthesizer import Synthesizer
from deepeval.dataset import Golden
from deepeval.synthesizer.config import StylingConfig
from repositories.faq import  get_all as get_all_faqs

faqs = get_all_faqs()
context_format = "Thông tin về câu hỏi và câu trả lời thường gặp của khách hàng tại FPT Shop:\nCâu hỏi: {question}\nCâu trả lời: {answer}\n"
contexts = [context_format.format(question=faq.question, answer=faq.answer) for faq in faqs]

styling_config = StylingConfig(
    input_format="Các câu hỏi bằng tiếng Việt liên quan đến sản phẩm, chính sách bảo hành, thanh toán, giao hàng, và dịch vụ tại FPT Shop.",
    expected_output_format="Câu trả lời tư vấn rõ ràng, chính xác, thể hiện sự chuyên nghiệp và thân thiện như một nhân viên bán hàng của FPT Shop.",
    task="Trả lời các câu hỏi thường gặp của khách hàng về sản phẩm và dịch vụ của FPT Shop nhằm hỗ trợ bán hàng và chăm sóc khách hàng.",
    scenario="Khách hàng tiềm năng hoặc khách hàng hiện tại đang cần được giải đáp nhanh chóng về thông tin mua sắm, chính sách và dịch vụ tại FPT Shop.",
)



In [5]:
synthesizer = Synthesizer(
    model="gpt-4o-mini",
    max_concurrent=10,
    cost_tracking=True,
    styling_config=styling_config,
)
# generate test set with gpt-4o-mini in goldens.json
goldens = synthesizer.generate_goldens_from_contexts(
    contexts=[[context] for context in contexts],
)

✨ Generating up to 68 goldens using DeepEval (using gpt-4o-mini, method=default): 100%|██████████| 68/68 [01:26<00:00,  1.27s/it]


💰 API cost: 0.037015

💰 API cost: 0.037015

In [1]:
from uuid import uuid4
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import  gen_answer

def get_actual_answer(input: str) -> str:
    user = create_user(CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user))
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    
    return gen_answer(
        thread_id=thread.id,
        history=[{"role": "user", "content": str(input)}],
        user_id=user.id,
    )
    

2025-05-03 10:26:22 - Loaded .env file
2025-05-03 10:26:26 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-05-03 10:26:27 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
2025-05-03 10:26:28 - file_cache is only supported with oauth2client<4.0.0


In [1]:
from typing import Callable, List, Optional
from deepeval.conversation_simulator import (
    ConversationSimulator,
)
import deepeval.models as deepeval_models
from deepeval.test_case.conversational_test_case import  ConversationalTestCase
from uuid import UUID, uuid4

import tqdm
from models.message import MessageType
from models.thread import ThreadModel
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer
from repositories.message import create as create_message, CreateMessageModel
from openai.types.chat import (
    ChatCompletionMessageParam,
    ChatCompletionAssistantMessageParam,
    ChatCompletionUserMessageParam,
)
from repositories.message import (
    create as create_message,
    get_by_fb_message_id as get_message_by_fb_message_id,
    get_all as get_all_messages,
)
from service.wandb import client as wandb_client

gpt_4o_mini = deepeval_models.GPTModel(
    model="gpt-4.1-mini",
    timeout=60,
)

async def a_gen_answer(
    thread_id: UUID,
    history: list[ChatCompletionMessageParam],
    user_id: UUID,
) -> str:
    return gen_answer(
        thread_id=thread_id,
        history=history,
        user_id=user_id,
    )

user_profile_items = ["first name", "last name", "Vietnamese Phone Number",]
user_intentions = [
    "asking about a samsung phone",
    "asking about a warranty policy",
    "asking about a payment method",
    "asking about a delivery time",
    "asking about a phone that have a price under 10 million VND",
]


class CustomConversationSimulator(ConversationSimulator):
    async def _a_simulate_single_conversation(
        self,
        model_callback: Callable,
        min_turns: int,
        max_turns: int,
        _progress_bar: Optional[tqdm.std.tqdm] = None,
    ) -> ConversationalTestCase:
        self.user = create_user(
            CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
        )
        self.thread = create_thread(
            CreateThreadModel(user_id=self.user.id, name=self.user.user_name)
        )
        single_conversation_simulation_call = wandb_client.create_call(
            op="single_conversation_simulation",
            inputs={"user_id": str(self.user.id), "thread_id": str(self.thread.id)},
        )
        test_case = await super()._a_simulate_single_conversation(
            model_callback=model_callback,
            min_turns=min_turns,
            max_turns=max_turns,
        )
        
        wandb_client.finish_call(
            single_conversation_simulation_call,
            output={
                "test_case": test_case.__dict__,
            },
        )
            
        
        return test_case

    async def _a_generate_chatbot_response(self, input: str, model_callback: Callable):
        message_data = CreateMessageModel(
            thread_id=self.thread.id,
            type=MessageType.user,
            content=str(input),
        )

        new_message = create_message(message_data)
        history = self.get_openai_message(self.thread)
        res = await model_callback(user_id=self.user.id, thread_id=self.thread.id, history=history)
        new_assistant_message_data = CreateMessageModel(
            thread_id=self.thread.id,
            type=MessageType.bot,
            content=str(res),
        )
        new_assistant_message = create_message(new_assistant_message_data)

        return str(res)

    def get_openai_message(
        self, thread: ThreadModel, limit: int = 10
    ) -> list[ChatCompletionMessageParam]:
        messages = get_all_messages(thread.id)
        return [
            (
                ChatCompletionUserMessageParam(
                    role="user",
                    content=message.content,
                )
                if message.type == MessageType.user
                else ChatCompletionAssistantMessageParam(
                    role="assistant",
                    content=message.content,
                )
            )
            for message in messages
        ]

    def simulate(
        self,
        model_callback: Callable = a_gen_answer,
        min_turns: int = 5,
        max_turns: int = 10,
        num_conversations: int = 1,
    ) -> List[ConversationalTestCase]:
        return super().simulate(model_callback, min_turns, max_turns, num_conversations)


convo_simulator = CustomConversationSimulator(
    user_profile_items=user_profile_items,
    user_intentions=user_intentions,
    simulator_model=gpt_4o_mini,
    async_mode=True,
)
convo_simulator.language = "Vietnamese"

test_cases = convo_simulator.simulate()

2025-05-09 22:52:51 - Loaded .env file
2025-05-09 22:52:55 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-05-09 22:52:55 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
weave version 0.51.45 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
2025-05-09 22:52:56 - file_cache is only supported with oauth2client<4.0.0


🪄 Simulating 1 conversational test case(s) using DeepEval (using gpt-4.1-mini):   0%|          | 0/1 [00:00<?, ?it/s]

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0196b5bf-efe6-7732-87ec-eab38def6669
2025-05-09 22:53:00 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-09 22:53:03 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-09 22:53:04 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-09 22:53:07 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User request: {'user_demand': <ProductType.MOBILE_PHONE: 'mobile phone'>, 'user_info': {'phone_number': None, 'email': None}}
Detect demand response: type='finished' content='The user request has been successfully processed.' instructions=[] UserIntent(is_user_needs_other_suggestions=False, product_type=<ProductType.MOBILE_PHONE: 'mobile phone'>)
2025-05-09 22:53:09 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Tool response for collect_and_update_phone_price_requirement

🪄 Simulating 1 conversational test case(s) using DeepEval (using gpt-4.1-mini):   0%|          | 0/1 [01:12<?, ?it/s]

Phone generate response: type='finished' content='Khi mua trả góp tại cửa hàng FPT Shop, bạn cần chuẩn bị một số giấy tờ và thủ tục như sau:\n\n1. **Giấy tờ cá nhân**:\n   - Chứng minh nhân dân hoặc thẻ căn cước công dân (bản gốc và bản photo).\n   - Sổ hộ khẩu (bản photo) nếu cần thiết.\n\n2. **Giấy tờ chứng minh thu nhập** (nếu có yêu cầu):\n   - Hợp đồng lao động hoặc giấy xác nhận lương từ công ty.\n   - Sao kê tài khoản ngân hàng trong 3 tháng gần nhất.\n\n3. **Điền thông tin vào đơn đăng ký trả góp**:\n   - Bạn sẽ cần điền thông tin cá nhân và thông tin sản phẩm muốn mua.\n\n4. **Thời gian xét duyệt**:\n   - Thời gian xét duyệt hồ sơ thường nhanh chóng, bạn có thể nhận được kết quả ngay trong ngày.\n\nNếu bạn cần thêm thông tin chi tiết hoặc có câu hỏi khác, hãy cho mình biết nhé! Để tư vấn tốt hơn cho bạn, bạn có thể cho mình biết số điện thoại hoặc email của bạn không?'


In [2]:
conversations = test_cases

conversation = conversations[0]  # Lấy cuộc hội thoại đầu tiên
metadata = conversation.additional_metadata
print(f"Hồ sơ người dùng: {metadata['User Profile']}")
print(f"Ý định người dùng: {metadata['User Intent']}")
print("\n=== NỘI DUNG HỘI THOẠI ===\n")

# In các lượt đối thoại
for i, turn in enumerate(conversation.turns):
    if i == 0 and turn.input == "":
        # Tin nhắn mở đầu từ chatbot
        print(f"Chatbot: {turn.actual_output}")
    else:
        print(f"Người dùng: {turn.input}")
        print(f"Chatbot: {turn.actual_output}")
    print()

Hồ sơ người dùng: Nguyễn Văn An là một cá nhân có thể liên hệ qua số điện thoại Việt Nam của mình.
Ý định người dùng: asking about a phone that have a price under 10 million VND

=== NỘI DUNG HỘI THOẠI ===

Người dùng: Chào bạn, tôi đang muốn mua một chiếc điện thoại mới với ngân sách dưới 10 triệu đồng. Bạn có thể tư vấn giúp tôi những mẫu nào phù hợp và các chương trình khuyến mãi hiện có không?
Chatbot: Chào bạn! Rất vui được hỗ trợ bạn trong việc chọn mua điện thoại mới. Để tư vấn chính xác hơn, bạn có thể cho tôi biết bạn thích thương hiệu nào không? Ví dụ như Samsung, Xiaomi, Oppo, hay iPhone?

Người dùng: Tôi ưu tiên các thương hiệu như Samsung hoặc Xiaomi, bạn có thể giới thiệu mẫu nào phù hợp không?
Chatbot: Hiện tại, FPT Shop có một số mẫu điện thoại Xiaomi phù hợp với ngân sách của bạn dưới 10 triệu đồng. Một trong những mẫu nổi bật là:

1. **[Xiaomi Redmi 14C](https://fptshop.com.vn/dien-thoai/xiaomi-redmi-14c)** 
   - Giá gốc: 3.290.000 VNĐ, Giá khuyến mãi: 3.190.000 VNĐ
 